<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [2]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)


float32 (200,)


In [3]:
print(f"Num of words: {len(wv_embeddings.index_to_key )}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [4]:
word_1 = 'dog'
word_2 = 'cat'
top=5
count=0
idxs = []

for word in wv_embeddings.most_similar(positive=[word_1], topn=top):
    if word_2 == word[0]:
        print(f'Слово {word_2} входит в топ-{top} к слову {word_1} и находится на {wv_embeddings.most_similar(positive=[word_1], topn = top).index(word) + 1} месте')
        break
    elif word_2 in word[0]:
        count+=1
        idxs.append(wv_embeddings.most_similar(positive=[word_1], topn = top).index(word) + 1)
    else:
        count += 1
if count == top:
    print(f'Слово "{word_2}" НЕ входит в топ-{top} близких к слову "{word_1}"')
if len(idxs) != 0:
    print(f'Однако, на {idxs[0]} позиции в топ-{top} находится слово "{wv_embeddings.most_similar(positive=[word_1], topn = top)[idxs[0]-1][0]}"')
print(wv_embeddings.most_similar(positive=[word_1], topn=top))


Слово "cat" НЕ входит в топ-5 близких к слову "dog"
Однако, на 4 позиции в топ-5 находится слово "cats"
[('animal', 0.8564180135726929), ('dogs', 0.7880867123603821), ('mammal', 0.7623804211616516), ('cats', 0.7621253728866577), ('animals', 0.760793924331665)]


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [5]:
import numpy as np
import re
from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

<>:8: SyntaxWarning: invalid escape sequence '\w'
<>:8: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Katie\AppData\Local\Temp\ipykernel_17924\3933099588.py:8: SyntaxWarning: invalid escape sequence '\w'
  return re.findall('\w+', text)


In [6]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    embed_list = []
    for word in tokenizer.tokenize(question.lower()):
        try:
            embed_list.append(embeddings[word])
        except (KeyError):
            continue
    return np.mean(embed_list, axis=0) if len(embed_list) > 0 else np.zeros(dim, dtype='float32')
    

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [7]:
text = "I love neural networks"
print(MyTokenizer().tokenize(text))
round(question_to_vec(text, wv_embeddings, tokenizer)[2],2)

['I', 'love', 'neural', 'networks']


-1.29

In [8]:
tk = WordPunctTokenizer()
print(tk.tokenize(text))
round(question_to_vec(text, wv_embeddings, tk)[2],2)

['I', 'love', 'neural', 'networks']


-1.29

In [9]:
import spacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')


class SpacyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        spacy_results = nlp(text)
        return [token.lemma_ for token in spacy_results]
    
spacy_tokenizer = SpacyTokenizer()
print(SpacyTokenizer().tokenize(text)) 
round(question_to_vec(text, wv_embeddings, spacy_tokenizer)[2],2)

['I', 'love', 'neural', 'network']


-1.41

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Максимальное значение 1.
Максимальное значение достигается в том случае, когда первое слагаемое (уменьшаемое) принимает максимальное значение, а второе (вычитаемое) принимает минимальное значение. Оба слагаемых могут принимать значения в интервале [0,1]. Следовательно, первое принимает 1 когда дубликаты вопросов расположены среди первых 47 вариантов ответов, в число которых могут входить и отрицательные примеры. Второе должно принимать 0. Это возможно в том случае, когда позиции дубликатов в ранжированном списке ближайших вопросов для вопроса 𝑞𝑖 не расположены на 1-ом месте.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [19]:
import math
K = 10
ranking = np.array([9])

DCG = round(sum(([np.zeros(1)[0] if rank > K else math.log2(1+rank)**(-1) for rank in ranking])) / ranking.shape[0],2)

print(f'\nDCG@{K} = {DCG}\n')



DCG@10 = 0.3



1

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [20]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = round(sum(([np.zeros(1)[0] if rank > k else np.ones(1)[0] for rank in dup_ranks])) / len(dup_ranks),3)
    return hits_value

In [25]:

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = round(sum([np.zeros(1)[0] if rank > k else math.log2(1+rank)**(-1) for rank in dup_ranks]) / len(dup_ranks),3)
    return dcg_value


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [22]:
import pandas as pd

In [26]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.631, 0.631, 0.631]


У вас должно получиться

In [27]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [30]:
!unzip stackoverflow_similar_questions.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


Считайте данные.

In [56]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [57]:
validation_data = read_corpus('./stackoverflow_similar_questions/data/validation.tsv')

Кол-во строк

In [58]:
len(validation_data)

3760

Размер нескольких первых строк

In [59]:
for i in range(10):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [60]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [71]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    result = []
    q_vec = question_to_vec(question, embeddings, tokenizer)
    
    for idx, elem in enumerate(candidates):
        candidate_vec = question_to_vec(elem, embeddings, tokenizer)
        result.append((idx, elem, cosine_similarity(q_vec.reshape(1, -1), candidate_vec.reshape(1, -1))[0][0]))
     
    result.sort(key=lambda x: x[2], reverse=True)
    
    return [elem[:-1] for elem in deepcopy(result)]

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [66]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [67]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

res [(0, 'Convert Google results object (pure js) to Python object', 0.5414996), (1, 'C# create cookie from string and send it', 0.5922089), (2, 'How to use jQuery AJAX for an outside domain?', 0.09703708)]
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

res [(0, 'Getting all list items of an unordered list in PHP', 0.46132383), (1, 'WPF- How to update the changes in list item of a list', 0.30059382), (2, 'select2 not displaying search results', 0.32301268)]
[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [68]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [69]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

In [74]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [76]:
train_data = read_corpus('./stackoverflow_similar_questions/data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [77]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
stopWords = set(stopwords.words('english'))

In [83]:
def preproc_nltk(text):
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])

In [84]:
couple_list = [couple[0] + str(' ') + couple[1] for couple in train_data]
words = [preproc_nltk(question).split() for question in couple_list]

In [123]:
couple_list[0]

'converting string to list Convert Google results object (pure js) to Python object\n'

In [91]:
words

[['converting',
  'string',
  'list',
  'convert',
  'google',
  'results',
  'object',
  '(',
  'pure',
  'js',
  ')',
  'python',
  'object'],
 ['html',
  '5',
  'canvas',
  'javascript',
  'use',
  'making',
  'interactive',
  'drawing',
  'tool',
  '?',
  'event',
  'handling',
  'geometries',
  'three.js',
  '?'],
 ['sending',
  'array',
  'via',
  'ajax',
  'fails',
  'getting',
  'list',
  'items',
  'unordered',
  'list',
  'php'],
 ['insert',
  'cookiecollection',
  'cookiecontainer',
  '?',
  'c',
  '#',
  'create',
  'cookie',
  'string',
  'send'],
 ['updating',
  'one',
  'element',
  'bound',
  'observable',
  'collection',
  'wpf-',
  'update',
  'changes',
  'list',
  'item',
  'list'],
 ['mongodb',
  'error',
  'find',
  '(',
  ')',
  'retrieve',
  'queried',
  'element',
  'object',
  'array',
  'mongodb',
  'collection'],
 ['select2',
  'displaying',
  'search',
  'results',
  'use',
  'jquery',
  'ajax',
  'outside',
  'domain',
  '?'],
 ['using',
  'reduce',
  'mer

In [111]:
dcg_history = []
hits_history = []
for min_count in range(7,10):
    for window in range(7,10):
        embeddings_trained = Word2Vec(words,                    # data for model to train on
                                      vector_size=200,          # embedding vector size
                                      min_count=min_count,      # Ignores all words with total frequency lower than this. Consider words that occured at least 5 times
                                      window=window,            # Maximum distance between the current and predicted word within a sentence
                                      workers=4,                # Use these many worker threads to train the model
                                      sg=1).wv                  # sg=0 - CBOW, sg=1 - Skip-Gram
        wv_ranking = []
        max_validation_examples = 1000
        tokenizer = MyTokenizer()
        for i, line in enumerate(validation_data):
            if i == max_validation_examples:
                break
            q, *ex = line
            ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
            wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        print(f'min_count = {min_count}, window = {window} with Skip-Gram')
        for k in tqdm([1, 5, 10, 100, 500, 1000]):
            print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
            dcg_history.append(dcg_score(wv_ranking, k))
            hits_history.append(hits_count(wv_ranking, k))
        print()

min_count = 7, window = 7 with Skip-Gram


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.523 | Hits@   1: 0.523
DCG@   5: 0.607 | Hits@   5: 0.679
DCG@  10: 0.629 | Hits@  10: 0.749
DCG@ 100: 0.665 | Hits@ 100: 0.924
DCG@ 500: 0.673 | Hits@ 500: 0.986
DCG@1000: 0.674 | Hits@1000: 1.000

min_count = 7, window = 8 with Skip-Gram


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.522 | Hits@   1: 0.522
DCG@   5: 0.609 | Hits@   5: 0.681
DCG@  10: 0.628 | Hits@  10: 0.741
DCG@ 100: 0.665 | Hits@ 100: 0.918
DCG@ 500: 0.674 | Hits@ 500: 0.985
DCG@1000: 0.676 | Hits@1000: 1.000

min_count = 7, window = 9 with Skip-Gram


...

DCG@   1: 0.524 | Hits@   1: 0.524
DCG@   5: 0.607 | Hits@   5: 0.678
DCG@  10: 0.626 | Hits@  10: 0.738
DCG@ 100: 0.663 | Hits@ 100: 0.912
DCG@ 500: 0.672 | Hits@ 500: 0.983
DCG@1000: 0.674 | Hits@1000: 1.000

min_count = 9, window = 8 with Skip-Gram


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.523 | Hits@   1: 0.523
DCG@   5: 0.611 | Hits@   5: 0.687
DCG@  10: 0.629 | Hits@  10: 0.742
DCG@ 100: 0.665 | Hits@ 100: 0.914
DCG@ 500: 0.675 | Hits@ 500: 0.986
DCG@1000: 0.676 | Hits@1000: 1.000

min_count = 9, window = 9 with Skip-Gram


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.523 | Hits@   1: 0.523
DCG@   5: 0.610 | Hits@   5: 0.685
DCG@  10: 0.629 | Hits@  10: 0.745
DCG@ 100: 0.665 | Hits@ 100: 0.915
DCG@ 500: 0.674 | Hits@ 500: 0.985
DCG@1000: 0.675 | Hits@1000: 1.000



In [113]:
columns = ['DCG@1','DCG@5','DCG@10','DCG@100','DCG@500','DCG@1000',
           'Hits@1','Hits@5','Hits@10','Hits@100','Hits@500','Hits@1000']
row_list = []
for i in range(7,10):
    for j in range(7,10):
        row_list.append('Min_'+str(i)+'/Win_'+str(j))

df = pd.DataFrame()
m = n = 0
for row in row_list:
    for dcg in columns[:6]:
        df.loc[row,dcg] = dcg_history[m]
        m+=1
    for hits in columns[6:]:
        df.loc[row,hits] = hits_history[n]
        n+=1

df['SUM'] = df.sum(axis=1)
df

,DCG@1,DCG@5,DCG@10,DCG@100,DCG@500,DCG@1000,Hits@1,Hits@5,Hits@10,Hits@100,Hits@500,Hits@1000,SUM
Min_7/Win_7,0.523,0.607,0.629,0.665,0.673,0.674,0.523,0.679,0.749,0.924,0.986,1.0,8.632
Min_7/Win_8,0.522,0.609,0.628,0.665,0.674,0.676,0.522,0.681,0.741,0.918,0.985,1.0,8.621
Min_7/Win_9,0.530,0.614,0.634,0.669,0.678,0.680,0.530,0.687,0.748,0.917,0.986,1.0,8.673
Min_8/Win_7,0.525,0.610,0.626,0.664,0.673,0.675,0.525,0.685,0.737,0.918,0.984,1.0,8.622
Min_8/Win_8,0.518,0.605,0.626,0.662,0.671,0.673,0.518,0.677,0.744,0.917,0.984,1.0,8.595
Min_8/Win_9,0.529,0.613,0.634,0.668,0.678,0.679,0.529,0.686,0.750,0.914,0.985,1.0,8.665
Min_9/Win_7,0.524,0.607,0.626,0.663,0.672,0.674,0.524,0.678,0.738,0.912,0.983,1.0,8.601
Min_9/Win_8,0.523,0.611,0.629,0.665,0.675,0.676,0.523,0.687,0.742,0.914,0.986,1.0,8.631
Min_9/Win_9,0.523,0.610,0.629,0.665,0.674,0.675,0.523,0.685,0.745,0.915,0.985,1.0,8.629


In [ ]:
df[df['Hits@1']==df['Hits@1'].max()] #max value of Hits@1

In [ ]:
df[df['SUM']==df['SUM'].max()] #max sum all metrics

choose min_count=9, window=10

In [116]:

embeddings_trained = Word2Vec(words, # data for model to train on
            vector_size=200,         # embedding vector size
            min_count=9,             # Ignores all words with total frequency lower than this. Consider words that occured at least 5 times
            window=10,               # Maximum distance between the current and predicted word within a sentence
            workers=4,               # Use these many worker threads to train the model
            sg=1).wv   

In [117]:
len(embeddings_trained)

29218

In [ ]:
tokenizers = [MyTokenizer(), SpacyTokenizer(), WordPunctTokenizer()]
count=0

for tokenizer in tokenizers:
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(validation_data):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    if count < 2:
        print(str(tokenizer).split(".")[1].split(" ")[0])
    else:
        print(str(tokenizer).split("(")[0])
    for k in [1, 5, 10, 100, 500, 1000]:
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    print()
    count+=1

MyTokenizer
DCG@   1: 0.530 | Hits@   1: 0.530
DCG@   5: 0.617 | Hits@   5: 0.691
DCG@  10: 0.636 | Hits@  10: 0.753
DCG@ 100: 0.670 | Hits@ 100: 0.912
DCG@ 500: 0.680 | Hits@ 500: 0.985
DCG@1000: 0.681 | Hits@1000: 1.000

эмбеддинги с нормализацией (лемматизацией)


In [119]:
def preproc_nltk_lemma(text):
    text = ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])
    return SpacyTokenizer().tokenize(text)
words_lemma = [preproc_nltk_lemma(question) for question in couple_list]

In [120]:
words_lemma

[['convert',
  'string',
  'list',
  'convert',
  'google',
  'result',
  'object',
  '(',
  'pure',
  'js',
  ')',
  'python',
  'object'],
 ['html',
  '5',
  'canvas',
  'javascript',
  'use',
  'make',
  'interactive',
  'drawing',
  'tool',
  '?',
  'event',
  'handle',
  'geometry',
  'three.js',
  '?'],
 ['send',
  'array',
  'via',
  'ajax',
  'fail',
  'get',
  'list',
  'item',
  'unordered',
  'list',
  'php'],
 ['insert',
  'cookiecollection',
  'cookiecontainer',
  '?',
  'c',
  '#',
  'create',
  'cookie',
  'string',
  'send'],
 ['update',
  'one',
  'element',
  'bind',
  'observable',
  'collection',
  'wpf-',
  'update',
  'change',
  'list',
  'item',
  'list'],
 ['mongodb',
  'error',
  'find',
  '(',
  ')',
  'retrieve',
  'query',
  'element',
  'object',
  'array',
  'mongodb',
  'collection'],
 ['select2',
  'display',
  'search',
  'result',
  'use',
  'jquery',
  'ajax',
  'outside',
  'domain',
  '?'],
 ['use',
  'reduce',
  'merge',
  'multiple',
  'datum',
 

In [121]:
embeddings_trained_lemma = Word2Vec(words_lemma, # data for model to train on
            vector_size=200,         # embedding vector size
            min_count=9,             # Ignores all words with total frequency lower than this. Consider words that occured at least 5 times
            window=10,               # Maximum distance between the current and predicted word within a sentence
            workers=4,               # Use these many worker threads to train the model
            sg=1).wv 

In [122]:
tokenizers = [MyTokenizer(), SpacyTokenizer(), WordPunctTokenizer()]
count=0


wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(validation_data):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained_lemma, MyTokenizer())
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
print()


DCG@   1: 0.359 | Hits@   1: 0.359
DCG@   5: 0.440 | Hits@   5: 0.510
DCG@  10: 0.460 | Hits@  10: 0.571
DCG@ 100: 0.500 | Hits@ 100: 0.772
DCG@ 500: 0.521 | Hits@ 500: 0.940
DCG@1000: 0.528 | Hits@1000: 1.000



### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


1. Лучшее качество дает токенизация, основанная на выражениях без применения нормализации, т.к. получается худшее ранжирование.
2. Не помогла и ухудшила метрики. 
3. Эмбеддинги без лематизации, с параметрами window = 10, min_count = 9.
4. Думаю, что плохое качество получилось из-за word2vec. Является устаревшим решением и модель много каких факторов не учитывает, например, плохо обрабатывает редкие слова. 
5. Возможно можно импользовать fastText. Данная библиотека способна генерировать эмбеддинги для неизвестных слов. 